In [ ]:
from tensorflow.python import keras
from matplotlib import pyplot as plt
from keras.src.utils import load_img, img_to_array
from keras.api.applications import resnet50
import numpy as np
import tensorflow as tf


# Load the pre-trained ResNet50 model
model = resnet50.ResNet50(weights='imagenet')

def preprocess_image(img_path, target_size=(224, 224)):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = resnet50.preprocess_input(img_array)
    return img_array

def deprocess_image(img):
    img = img.copy()
    img /= 2.
    img += 0.5
    img *= 255.
    img = np.clip(img, 0, 255).astype('uint8')
    return img

def visualize_layer_filters(img_array, layer_name, num_filters=64):
    layer_output = model.get_layer(layer_name).output
    feature_extractor = keras.Model(inputs=model.input, outputs=layer_output)
    
    features = feature_extractor.predict(img_array)
    features = features[0]
    
    num_rows = int(np.sqrt(num_filters))
    num_cols = int(np.ceil(num_filters / num_rows))
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20))
    fig.suptitle(f'Filters in layer: {layer_name}', fontsize=16)
    
    for i in range(num_filters):
        row = i // num_cols
        col = i % num_cols
        filter_img = features[:, :, i]
        axes[row, col].imshow(filter_img, cmap='viridis')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

def predict_image(img_array):
    preds = model.predict(img_array)
    decoded_preds = resnet50.decode_predictions(preds, top=3)[0]
    
    print("Top 3 predictions:")
    for i, (imagenet_id, label, score) in enumerate(decoded_preds):
        print(f"{i + 1}: {label} ({score:.2f})")

def visualize_and_predict(img_path):
    # Preprocess the image
    img_array = preprocess_image(img_path)
    
    # Visualize layers
    layers_to_visualize = [
        'conv1_conv',        # Early layer
        'conv2_block3_out',  # Early-middle layer
        'conv3_block4_out',  # Middle layer
        'conv5_block3_out'   # Late layer
    ]
    
    for layer_name in layers_to_visualize:
        visualize_layer_filters(img_array, layer_name)
    
    # Make prediction
    predict_image(img_array)
    
    # Display the original image
    original_img = load_img(img_path, target_size=(224, 224))
    plt.figure(figsize=(10, 10))
    plt.imshow(original_img)
    plt.title('Original Image')
    plt.axis('off')
    plt.show()

# Example usage
img_path = 'path/to/your/image.jpg'  # Replace with the path to your image
visualize_and_predict(img_path)